In [1]:
# FACES BASIC 1  FACES BASIC 1  FACES BASIC 1 
# REGION OF INTEREST (roi)
# REF VIDEO: https://www.codingforentrepreneurs.com/blog/opencv-python-face-recognition-and-identification

In [2]:
# Import for Capture Photo from video 
import cv2
import os
import shutil
# Import for Augumentation 
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
#import os
import matplotlib.pyplot as plt
# Import all necessary libaries
#import os
import numpy as np
from PIL import Image
#import cv2
import pickle

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
os.chdir('/Users/NidhiAneja/Documents/AI/Justin/OpenCV-Python-Series-master/src') 
BASE_DIR = '/Users/NidhiAneja/Documents/AI/Justin/OpenCV-Python-Series-master/src'
base_dir_images = '/Users/NidhiAneja/Documents/AI/Justin/OpenCV-Python-Series-master/src/images/'


In [4]:
# Model already built for faces -- CASCADE
face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')

In [5]:
# Into the folder of images
image_dir = os.path.join(BASE_DIR, "images")

In [6]:
#--------------------------
#Why does it alawys show directory of ".DS_Store"
#--------------------------
if not os.path.exists('Kunal'):
    os.makedirs('Kunal')
    
#os.listdir()

In [7]:
# these images are name of individual and than photos are numbered 1,2,3....

In [8]:

#---
BASE_DIR = os.path.dirname(os.path.abspath('src'))
# BASE_DIR
#---
face_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_frontalface_alt2.xml')
#---
y_labels= []
x_train = []

In [9]:
# CREATE DICTIONARY
current_id=0
label_ids= {}

In [10]:
# TRAIN THE RECOGINIZER....PART 3
# recognizer = cv2.face.LBPHFaceRecognizer

In [11]:
for root, dirs, files in os.walk(image_dir):
    for file in files:
        if file.endswith('png') or file.endswith('jpg'):
            path = os.path.join(root, file)
            label = os.path.basename(root).replace(" ","-").lower()
            #print(label,path)
            #----
            if label in label_ids:
                pass
            else:
                label_ids[label] = current_id
                current_id += 1
                
            id_ = label_ids[label]
            #print(label_ids)
                #----
            #y_labels.append(label) # some number
            #x_train.append(path) # verify this image, turn into a NUMPY array, GARY
            # PIL is a image library
            pil_image = Image.open(path).convert("L") # grayscale
            #---
            # RESIZE IMAGES IN TRAINING.....
            size = (550, 550)
            final_image = pil_image.resize(size, Image.ANTIALIAS)
            #---
            #image_array = np.array(pil_image, "uint8")
            image_array = np.array(final_image, "uint8")
            #print(image_array)
            # following gives region of interest i.e. faces...
            faces = face_cascade.detectMultiScale(image_array, scaleFactor=1.5, minNeighbors=5)
            
            for (x,y,w,h) in faces:
                roi = image_array[y:y+h, x:x+w]
                x_train.append(roi)
                y_labels.append(id_)
                
#print(y_labels)
#print(x_train)

with open("labels.pickle", 'wb') as f:
    pickle.dump(label_ids, f)
    

In [12]:
# TRAIN THE RECOGINIZER....PART 3
# recognizer = cv2.face.LBPHFaceRecognizer
recognizer = cv2.face.LBPHFaceRecognizer_create()
#----
#ACTUALLY TRAINING DONE HERE>...
recognizer.train(x_train, np.array(y_labels))
recognizer.save("trainner.yml") # this is how you save it
#----
recognizer.read("trainner.yml") # READ THIS which was trained earlier...
#----
labels = {}
with open("labels.pickle", 'rb') as f: # ADD PICKLE TO NAME THE PERSON IN PREDICTION...
    og_labels = pickle.load(f)
    labels = {v:k for k,v in og_labels.items()}    
#----


In [13]:
os.chdir(os.path.join(base_dir_images, 'Kunal'))
!pwd

/Users/NidhiAneja/Documents/AI/Justin/OpenCV-Python-Series-master/src/images/kunal


In [14]:
#========================
# EYES AND SMILE......
#========================
# EYE
eye_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_eye.xml')
# add the following to ready to predict code..
'''
eyes = eye_cascade.detectMultiScale(roi_gray)
for (ex, ey, ew, eh) in eyes:
    cv2.rectangle(roi_color, (ex,ey), (ex + ew, ey + eh), (0,255,0),2)
'''
# SMILE
smile_cascade = cv2.CascadeClassifier('cascades/data/haarcascade_smile.xml')
'''
smile = smile_cascade.detectMultiScale(roi_gray)
for (sx, sy, ew, eh) in eyes:
    cv2.rectangle(roi_color, (ex,ey), (ex + ew, ey + eh), (0,255,255),2)
    
'''


'\nsmile = smile_cascade.detectMultiScale(roi_gray)\nfor (sx, sy, ew, eh) in eyes:\n    cv2.rectangle(roi_color, (ex,ey), (ex + ew, ey + eh), (0,255,255),2)\n    \n'

In [15]:
# ready to predict....

cap = cv2.VideoCapture(0)
cv2.namedWindow("Who are you?")
img_counter = 0
j=0
while(True):
    
    # Capture frame-by-frame
    ret, frame = cap.read()
    cv2.imshow("Who are you?", frame)
    gray  = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)
    for (x, y, w, h) in faces:
        #print(x,y,w,h)
        roi_gray = gray[y:y+h, x:x+w]
        #print (roi_gray)
        # ADD A FRAME TO THE FACE CAPTURE
        roi_color = frame[y:y+h, x:x+w]        
        # recognize?
        id_, conf = recognizer.predict(roi_gray) # give label id and confidence
        if conf >= 95:
            print(id_)
            print(labels[id_])
            font = cv2.FONT_HERSHEY_SIMPLEX
            name = labels[id_]
            color = (255,255,255)
            stroke = 2
            cv2.putText(frame, name, (x,y), font,1, color, stroke, cv2.LINE_AA)
        #print(id_)
        print(labels[id_])
        font = cv2.FONT_HERSHEY_SIMPLEX
        name = labels[id_]
        color = (255,255,255)
        stroke = 2
        cv2.putText(frame, name, (x,y), font,1, color, stroke, cv2.LINE_AA)
        #-------------------------------------------------------
         
        '''COMPLETE_TEST_DIR = os.path.join(base_dir_images, labels[id_])
        #print(COMPLETE_TEST_DIR)
        for the_pics in os.listdir(COMPLETE_TEST_DIR):
            img_counter += 1
        #labels[id_]
        '''
        #-------------------------------------------------------
        '''while labels[id_] == labels[id_]:
            if labels
            j += 1
            print(j)'''
        if labels[id_] == 'kunal':
            j += 1
            if labels[id_] != 'kunal':
                j = 0
                
        
        
        #-------------------------------------------------------
        img_item = "my-image_TEST5.png"
        #cv2.imwrite(img_item, roi_gray)
        cv2.imwrite(img_item, roi_color)
        
        color = (255,0,0) # BGR 0-255
        stroke = 2
        end_cord_x = x + w
        end_cord_y = y + h
        cv2.rectangle(frame, (x,y), (end_cord_x, end_cord_y), color, stroke)
        '''
        #---- eyes
        eyes = eye_cascade.detectMultiScale(roi_gray)
        for (ex, ey, ew, eh) in eyes:
            cv2.rectangle(roi_color, (ex,ey), (ex + ew, ey + eh), (0,255,0),2)
        #----
        #---- smile
        smile = smile_cascade.detectMultiScale(roi_gray)
        for (sx, sy, sw, sh) in smile:
            cv2.rectangle(roi_color, (sx,sy), (sx + sw, sy + sh), (244, 203, 156),1)
        #print (smile)
        #print (smile.shape)
        # print ("Number of smiles detected: " + str(smile.shape[0]))
        #if smile.shape[0] > 3:
        #    print("5")
        #----
        '''
    #------------------------------------------------------
        
        
        k = cv2.waitKey(1)
        if k%256 == 32:
            # SPACE pressed
            #img_name = "opencv_frame_{}.png".format(img_counter)
            img_name = "{}.png".format(img_counter)
            cv2.imwrite(img_name, frame)
            print("{} written!".format(img_name))
            img_counter += 1
        # Display the resulting frame 
        #cv2.imshow('frame', frame)
        if cv2.waitKey(2) & 0xFF == ord('q'):
            break
        test = cap.get(cv2.CAP_PROP_FPS)
        test
        #just shows the frames at 12?
        #why?????????
        
        
        
        '''
        if not ret:
            break
        k = cv2.waitKey(1)
        if k%256 == 27:
            # ESC pressed
            print("Escape hit, closing...")
            break
        elif k%256 == 32:
            # SPACE pressed
            #img_name = "opencv_frame_{}.png".format(img_counter)
            img_name = "{}.png".format(img_counter)
            cv2.imwrite(img_name, frame)
            print("{} written!".format(img_name))
            img_counter += 1
        # Display the resulting frame 
        cv2.imshow('frame', frame)
        if cv2.waitKey(2) & 0xFF == ord('q'):
            break
        test = cap.get(cv2.CAP_PROP_FPS)
        test
        '''
        

kunal
1
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal
kunal


KeyboardInterrupt: 

In [16]:
cap.release()
cv2.destroyAllWindows()

